In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16 #to import model from ImageNet we can any other model to if we want
from keras.applications.vgg16 import preprocess_input #to import existing weights and all that
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
# because vgg16 accepts only size 224,224
IMAGE_SIZE = [224, 224]

#training path
train_path = 'dataset/train'
valid_path = 'dataset/test'

# add preprocessing layer to the front of VGG
#here by adding +3 we add channel
#include_top means we want to drop output of this pretrained model bcz it was trained for thousand of classification
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# here weights are already assigned we don't want to retrain our model again
for layer in vgg.layers:
  layer.trainable = False
  

  
 # this function is useful for getting number of classes in train folder
folders = glob('dataset/train/*')
  

# our layers - you can add more if you want, so here we are flattening output of vgg
x = Flatten()(vgg.output)
#using below code we are adding one more layer to our output
# x = Dense(1000, activation='relu')(x)

#here we are creating final layer which nodes are equals to number of folders
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 30,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 30,
                                            class_mode = 'categorical')

'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''

# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')






In [ ]:
#plot the accuracy graph
loss_train = r.history['acc']
loss_val = r.history['val_acc']
epochs = range(1,6)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf
from keras.models import load_model
model.save('facerecognition_model.h5')